<a href="https://colab.research.google.com/github/arutk1/Beijing_pollution/blob/main/Beijing_pollution_regression_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Until now, different models used to predict PM2.5 levels were not performing well. I will use the same dataset where the outliers have been removed and missing values have been dropped. And let's see the results.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import datetime
import seaborn as sns
import glob

In [ ]:
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 'PRSA_Data_Changping_20130301-20170228.csv',
 'PRSA_Data_Dingling_20130301-20170228.csv',
 'PRSA_Data_Shunyi_20130301-20170228.csv',
 'PRSA_Data_Aotizhongxin_20130301-20170228.csv',
 'PRSA_Data_Guanyuan_20130301-20170228.csv',
 'PRSA_Data_Wanliu_20130301-20170228.csv',
 'PRSA_Data_Huairou_20130301-20170228.csv',
 'PRSA_Data_Gucheng_20130301-20170228.csv',
 'PRSA_Data_Tiantan_20130301-20170228.csv',
 'PRSA_Data_Wanshouxigong_20130301-20170228.csv',
 'PRSA_Data_Dongsi_20130301-20170228.csv']

In [ ]:
# Concatenation of all files

data = pd.concat([pd.read_csv(f).drop(["No"],axis=1) for f in csv_files ], ignore_index=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   year     420768 non-null  int64  
 1   month    420768 non-null  int64  
 2   day      420768 non-null  int64  
 3   hour     420768 non-null  int64  
 4   PM2.5    412029 non-null  float64
 5   PM10     414319 non-null  float64
 6   SO2      411747 non-null  float64
 7   NO2      408652 non-null  float64
 8   CO       400067 non-null  float64
 9   O3       407491 non-null  float64
 10  TEMP     420370 non-null  float64
 11  PRES     420375 non-null  float64
 12  DEWP     420365 non-null  float64
 13  RAIN     420378 non-null  float64
 14  wd       418946 non-null  object 
 15  WSPM     420450 non-null  float64
 16  station  420768 non-null  object 
dtypes: float64(11), int64(4), object(2)
memory usage: 54.6+ MB


In [ ]:
# Checking for missing values

missing_values = data.isnull().sum().sort_values(ascending=False)

In [ ]:
# Calculating the percentage of missing data in each column
missing_percentage = round((missing_values / len(data)) * 100,2)

# Display the missing data statistics
print("Missing values in each column:\n", missing_values)
print("\nPercentage of missing data:\n", missing_percentage)

Missing values in each column:
 CO         20701
O3         13277
NO2        12116
SO2         9021
PM2.5       8739
PM10        6449
wd          1822
DEWP         403
TEMP         398
PRES         393
RAIN         390
WSPM         318
year           0
month          0
hour           0
day            0
station        0
dtype: int64

Percentage of missing data:
 CO         4.92
O3         3.16
NO2        2.88
SO2        2.14
PM2.5      2.08
PM10       1.53
wd         0.43
DEWP       0.10
TEMP       0.09
PRES       0.09
RAIN       0.09
WSPM       0.08
year       0.00
month      0.00
hour       0.00
day        0.00
station    0.00
dtype: float64


In [ ]:
# Remove rows with missing values
data.dropna(subset=['PM2.5'],inplace=True)

In [ ]:
# Verify that missing values have been removed
data.isnull().sum()

year           0
month          0
day            0
hour           0
PM2.5          0
PM10         216
SO2         3698
NO2         6747
CO         15162
O3          8145
TEMP         398
PRES         393
DEWP         403
RAIN         390
wd          1797
WSPM         317
station        0
dtype: int64

In [ ]:
# Removing outliers for PM2.5

Q1 = data['PM2.5'].quantile(0.25)

print("Q1:", Q1)

Q3 = data['PM2.5'].quantile(0.75)

print("Q3:", Q3)
IQR = Q3 - Q1
print("IQR:", IQR)

# Define the bounds for the outliers
lower_bound = Q1 - 1.5 * IQR
print("Lower bound:", lower_bound)
upper_bound = Q3 + 1.5 * IQR
print("Upper bound:", upper_bound)

# Remove outliers
data_no_outliers = data[(data['PM2.5'] >= lower_bound) & (data['PM2.5'] <= upper_bound)]

# Check the shape of the data before and after removal of outliers

print("Original data shape:", data.shape)
print("New data shape without outliers:", data_no_outliers.shape)

Q1: 20.0
Q3: 111.0
IQR: 91.0
Lower bound: -116.5
Upper bound: 247.5
Original data shape: (412029, 17)
New data shape without outliers: (392887, 17)


In [ ]:
# First, we need to get rid of missing values for our independant variables, otherwise the model will throw an error

data_no_outliers = data_no_outliers.loc[data_no_outliers['TEMP'].notna() &
                                        data_no_outliers['PRES'].notna() &
                                        data_no_outliers['RAIN'].notna() &
                                        data_no_outliers['WSPM'].notna()]




In [ ]:
# Defining features (independant variables) and target (dependant variable)

features = ['TEMP','PRES','RAIN','WSPM']
target = ['PM2.5']

X = data_no_outliers[features]
y = data_no_outliers[target]

In [ ]:
# For the sake of simplicity I will not use the validation dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1111)

# random_state = 1111 so the resulting split into training and test sets will be identical each time we run the code

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

Training set size: 313989 samples
Test set size: 78498 samples


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(random_state=42)

In [ ]:
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 2572.159384180981
Root Mean Squared Error: 50.7164606827111


After using Decision Trees Regressor, we can see that MSE and RMSE has the smallest value so far then in all cases before. This can be explained by the fact that the correlation between target and features is not linear. That is why the Linear Regression was not a good choice and was giving big MSE and RMSE.

# Random Forest Regressor

The data is already splitted into training and testing datasets. I will use the same datasets in a Random Forest Regressor model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(random_state=42)

In [ ]:
y_train.shape

(313989, 1)

In [ ]:
# y_train needs to be reshaped because for sklearn y should be an 1D array

y_train = y_train.to_numpy().reshape(-1)
y_train.shape

(313989,)

In [ ]:
rfr.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred = rfr.predict(X_test)

In [ ]:
mse_rfr = mean_squared_error(y_test, y_pred)
rmse_rfr = sqrt(mse_rfr)
print(f'Mean Squared Error: {mse_rfr}')
print(f'Root Mean Squared Error: {rmse_rfr}')

Mean Squared Error: 1733.0237204231478
Root Mean Squared Error: 41.62960149248546


Here, the MSE and RMSE is the smallest compared to other models I have verified. So far, I have used many ML models to predict PM2.5 pollution and for each of them I have calculated the MSE and RMSE. The algorithms I have used so far are: Linear Regression, Linear Regression using Neural Net, Neural Net and Neural Net for multiple inputs. Also Decision Trees Regressor and Random Forests Regressor.

Now I will try to improve my Random Forest Regressor. I didn't do that with Decision Trees Regressor because the MSE and RMSE are smaller for Random Forest Regressor so I want to improve this model with tuning hyperparameters.

In [ ]:
params = rfr.get_params()
print(f"These are the parameters used in this Random Forest Regression:\n {params}")

These are the parameters used in this Random Forest Regression:
 {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


Here, the algorith used 100 estimators and min samples leaf equal to 1 and min samples split equal to 2. Now I will try to improve the model by setting hyperparametres as below and looking for best hyperparameters using Grid Search CV and Randomized Search CV.

In [ ]:
param_grid = {
    'n_estimators': [30, 40, 50],
    'max_depth': [30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

In [ ]:
# with GridsearchCV

from sklearn.model_selection import GridSearchCV

In [ ]:
# rfr_cv = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=3, scoring ='neg_mean_squared_error')
# rfr_cv.fit(X_train, y_train)

Unfortunately, fitting Random Forest Regressor with variable rfr_cv is computationally too expensive, so I will try to use RandomizedSearchCV instead of GridSearchCV, which is less computationally expensive but it does not evaluate all combinations.

In [ ]:
# with RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

rfr_RandomGrid = RandomizedSearchCV(estimator=rfr, param_distributions=param_grid, cv = 3, verbose = 2, n_jobs = -1, n_iter = 10)

In [ ]:
rfr_RandomGrid.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [30, 40, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [30, 40, 50]},
                   verbose=2)

In [ ]:
# Print the best parameters

print("Best parameters found: ", rfr_RandomGrid.best_params_)

Best parameters found:  {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 40}


In [ ]:
y_pred = rfr_RandomGrid.predict(X_test)

In [ ]:
mse_rfr_RandomGrid = mean_squared_error(y_test, y_pred)
rmse_rfr_RandomGrid = sqrt(mse_rfr_RandomGrid)
print(f'Mean Squared Error: {mse_rfr_RandomGrid}')
print(f'Root Mean Squared Error: {rmse_rfr_RandomGrid}')

Mean Squared Error: 1768.473300234532
Root Mean Squared Error: 42.053219855731996


What is interesting here, that after trying various hyperparameters combinations, it looks like the number of trees has the biggest impact on the model's performance. But the MSE and RMSE is still slightly bigger than for Random Forest Regressor without hyperparamters tuning. Now I will try bigger number of trees in Random Forest and I will see what happens.

In [ ]:
param_grid = {
    'n_estimators': [50, 60, 70],
    'max_depth': [30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

In [ ]:
# with RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

rfr_RandomGrid = RandomizedSearchCV(estimator=rfr, param_distributions=param_grid, cv = 3, verbose = 2, n_jobs = -1, n_iter = 10)

In [ ]:
rfr_RandomGrid.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [30, 40, 50],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 60, 70]},
                   verbose=2)

In [49]:
# Print the best parameters

print("Best parameters found: ", rfr_RandomGrid.best_params_)

Best parameters found:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 50}


In [50]:
y_pred = rfr_RandomGrid.predict(X_test)

In [51]:
mse_rfr_RandomGrid = mean_squared_error(y_test, y_pred)
rmse_rfr_RandomGrid = sqrt(mse_rfr_RandomGrid)
print(f'Mean Squared Error: {mse_rfr_RandomGrid}')
print(f'Root Mean Squared Error: {rmse_rfr_RandomGrid}')

Mean Squared Error: 1746.541341372459
Root Mean Squared Error: 41.79164200378419


The MSE and RMSE are so far the lowest for Random Forest Regressor.


# Gradient Boosting Regression


Boosting in ML means that we are combining multiple models together to have better results.

In [52]:
from sklearn.ensemble import GradientBoostingRegressor

In [53]:
gbr = GradientBoostingRegressor()

In [54]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [55]:
y_pred = gbr.predict(X_test)

In [56]:
mse_gbr = mean_squared_error(y_test, y_pred)
rmse_gbr = sqrt(mse_gbr)
print(f'Mean Squared Error: {mse_gbr}')
print(f'Root Mean Squared Error: {rmse_gbr}')

Mean Squared Error: 2861.931747922374
Root Mean Squared Error: 53.49702559883469


# XGBoost Regressor

In [57]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred)
rmse_xgb = sqrt(mse_xgb)

print(f"Mean Squared Error: {mse_xgb}")
print(f"Root Mean Squared Error: {rmse_xgb}")


Mean Squared Error: 2747.578732964545
Root Mean Squared Error: 52.417351449348764


Summary: the smallest MSE and RMSE was obtained using Random Forest Regressor.